In [ ]:
import pandas as pd
import numpy as np
import os
import utils

In [ ]:
features_1 = [
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]



In [ ]:
path = "./tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [ ]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
print(df['label'].value_counts())
df.head()

In [ ]:
lengths = []
for i in range(len(df)):
    lengths.append(df.iloc[i]['time_series'].shape[0])
print(np.mean(lengths))

In [ ]:
import sklearn as sk
from sklearn.model_selection import train_test_split
seed = 0

# Shuffle the dataframe
shuffled_df = sk.utils.shuffle(df, random_state=seed)
X_train, X_test, y_train, y_test = train_test_split(shuffled_df['time_series'], shuffled_df['label'], stratify=shuffled_df['label'],  train_size=.9, random_state=seed)

In [ ]:
unique, counts = np.unique(y_train.values, return_counts=True)
print(np.asarray((unique, counts)).T)

In [ ]:
from utils import compute_mean_feature_vector
X_train_mean_fv = compute_mean_feature_vector(X_train)
X_test_mean_fv = compute_mean_feature_vector(X_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

SC = StandardScaler()
SC.fit(X_train_mean_fv)
matrix_X_train_mean_fv_scaled = SC.transform(X_train_mean_fv)
matrix_X_test_mean_fv_scaled = SC.transform(X_test_mean_fv)



In [ ]:
import logging
logging.basicConfig(filename="logfilename.log", level=logging.INFO)


In [ ]:
##MAnca il k fold (forse non lo teniamo)

import itertools
from tqdm import tqdm
best_acc_score = 0
best_combination = ''
params = [['linear', 'poly', 'rbf', 'sigmoid'], [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 2, 5, 10], [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 2, 5, 10],[2, 3, 5, 6, 8, 9, 11, 13, 15, 17, 20, 25, 30]]
params_comb = list(itertools.product(*params))
max_iter=100
for params in tqdm(params_comb, desc='calculating diff params combinations..'):
    try:
        svc = SVC(kernel=params[0], C=params[1], gamma = params[2], degree= params[3], max_iter=max_iter)
        svc.fit(matrix_X_train_mean_fv_scaled, y_train.values)
        predictions = svc.predict(matrix_X_test_mean_fv_scaled)
        res = accuracy_score(y_test.values, predictions)
        if res > best_acc_score:
            best_acc_score = res
            best_combination = f'kernel:{params[0]}, C={params[1]}, gamma={params[2]}, degree={params[3]}.'
            logging.info(f'[SCORE] found new best acc score! {best_acc_score}, best combination: {best_combination}')
    except:
        logging.info(f'[SKIP] skipping the following parameters: kernel={params[0]}, C={params[1]}, gamma={params[2]}, degree={params[3]}')
        continue
print(f'best accuracy score: {best_acc_score}, best combination: {best_combination}')